In [1]:
# %pip install -q -U anthropic
# %pip install -q -U lightning
# %pip install -q -U bitsandbytes
# %pip install -q -U transformers
# %pip install -q -U peft
# %pip install -q -U accelerate
# %pip install -q -U wandb
# %pip install -q -U datasets
# %pip install -q -U prodigyopt
# %pip install pdf2image
# %pip install -q -U protobuf==3.20.0
#!apt-get update --yes
#!apt-get install -y poppler-utils

In [2]:
import torch
import os
import json
import random
from transformers import AutoProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from transformers import DonutProcessor
from torch.utils.data import DataLoader
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import os, torch, torch.nn as nn, torch.utils.data as data, torchvision as tv
import lightning as L
from transformers import DonutProcessor, VisionEncoderDecoderModel
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
import wandb
import os
from IPython.display import Image
from tqdm.auto import tqdm
from IPython.display import Image
import pickle
import torchvision.transforms as T
from datasets import Dataset
from datasets import IterableDataset
from tqdm.auto import tqdm
import unicodedata
import os
import gc
from torch.utils.data import Dataset
import torchvision as tv
import torchvision.transforms as T
from transformers import DonutProcessor
import random
from torch.utils.data import Subset
from peft import LoraConfig
from torch.optim.lr_scheduler import LinearLR
import lightning as L
from prodigyopt import Prodigy
from datetime import datetime
import torch.multiprocessing as mp
from torch.cuda.amp import autocast, GradScaler
from pdf2image import convert_from_path, convert_from_bytes
from transformers import VisionEncoderDecoderModel, DonutProcessor
from pytorch_lightning.loggers import WandbLogger
from pdf2image import convert_from_path
from torchvision.transforms import ToTensor

2024-10-01 14:28:46.877361: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 14:28:46.915624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 14:28:46.915636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 14:28:46.916714: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 14:28:46.923189: I tensorflow/core/platform/cpu_feature_guar

In [3]:
DATASET_PATH = "./documents"
PDF_PATH = os.path.join(DATASET_PATH, "german_pdf_files")
IMAGE_PATH = os.path.join(DATASET_PATH, "german_img_files")


class DocMeta:
    def __init__(self, dataset_path=DATASET_PATH, image_path=IMAGE_PATH):
        self.dataset_path = dataset_path
        self.image_path = image_path
        self.extraction = self.load_extraction(self.dataset_path)
        self.mapping = self.load_mapping(self.image_path)

    def get_random_urls(self, num=100):
        return random.sample(list(self.extraction.keys()), num)

    def load_extraction(self, path):
        with open(os.path.join(path, "extraction.json"), "r") as f:
            return json.load(f)

    def load_mapping(self, path):
        with open(os.path.join(path, "mapping.json"), "r") as f:
            return json.load(f)

    def get_image_path(self, url):
        return os.path.join(self.image_path, self.mapping[url])

    def get_image(self, url):
        img_path = self.get_image_path(url)
        return Image(filename=img_path)


doc_meta = DocMeta()

In [4]:
new_special_tokens = []  # new tokens which will be added to the tokenizer
task_start_token = "<s>"  # start of task token
eos_token = "</s>"  # eos token of tokenizer


def json2token(
    obj, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True
):
    """
    Convert an ordered JSON object into a token sequence
    """
    if type(obj) == dict:
        if len(obj) == 1 and "text_sequence" in obj:
            return obj["text_sequence"]
        else:
            output = ""
            if sort_json_key:
                keys = sorted(obj.keys(), reverse=True)
            else:
                keys = obj.keys()
            for k in keys:
                if update_special_tokens_for_json_key:
                    (
                        new_special_tokens.append(rf"<s_{k}>")
                        if rf"<s_{k}>" not in new_special_tokens
                        else None
                    )
                    (
                        new_special_tokens.append(rf"</s_{k}>")
                        if rf"</s_{k}>" not in new_special_tokens
                        else None
                    )
                output += (
                    rf"<s_{k}>"
                    + json2token(
                        obj[k], update_special_tokens_for_json_key, sort_json_key
                    )
                    + rf"</s_{k}>"
                )
            return output
    elif type(obj) == list:
        return r"<sep/>".join(
            [
                json2token(item, update_special_tokens_for_json_key, sort_json_key)
                for item in obj
            ]
        )
    else:
        # excluded special tokens for now
        obj = str(obj)
        if f"<{obj}/>" in new_special_tokens:
            obj = f"<{obj}/>"  # for categorical special tokens
        return obj

In [5]:
def filter_metadata(meta):
    if meta["main_author"] == None:
        return True
    if meta["recipients"] == None:
        return True
    if meta["date"] == None:
        return True
    if meta["logo_owners"] == None:
        return True
    if meta["other_authors"] == None:
        return True


#    if not meta["has_signature"]:
#        return True

In [ ]:
# cnt = 0
# for key in dataset.doc_meta.extraction:
#    metadata = dataset.doc_meta.extraction[key]["metadata"]
#    if filter_metadata(metadata):
#        cnt += 1
#        continue
# print(cnt)

In [7]:
class DocMetaDataset(Dataset):
    def __init__(self, doc_meta, output_dir):
        self.doc_meta = doc_meta
        self.task_start_token = "<s>"
        self.eos_token = "</s>"
        self.processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
        self.processor.tokenizer.add_special_tokens(
            {
                "additional_special_tokens": [
                    self.task_start_token,
                    self.eos_token,
                    "<s_test-key>",
                    "</s_test-key>",
                    "<s_title>",
                    "</s_title>",
                    "<s_recipients>",
                    "</s_recipients>",
                    "<s_other_authors>",
                    "</s_other_authors>",
                    "<s_main_author>",
                    "</s_main_author>",
                    "<s_website>",
                    "</s_website>",
                    "<s_type>",
                    "</s_type>",
                    "<s_sector>",
                    "</s_sector>",
                    "<s_phone>",
                    "</s_phone>",
                    "<s_name>",
                    "</s_name>",
                    "<s_fax>",
                    "</s_fax>",
                    "<s_email>",
                    "</s_email>",
                    "<s_address>",
                    "</s_address>",
                    "<s_logo_owners>",
                    "</s_logo_owners>",
                    "<s_has_signature>",
                    "</s_has_signature>",
                    "<s_date>",
                    "</s_date>",
                    "<s_content_type>",
                    "</s_content_type>",
                    "<s_role>",
                    "</s_role>",
                    "<s_mobile>",
                    "</s_mobile>",
                    "<s_last_name>",
                    "</s_last_name>",
                    "<s_first_name>",
                    "</s_first_name>",
                    "<s_state>",
                    "</s_state>",
                    "<s_district>",
                    "</s_district>",
                    "<s_department>",
                    "</s_department>",
                    "<s_city>",
                    "</s_city>",
                ]
            }
        )
        self.processor.feature_extractor.size = [2160, 3840]
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
        self.data = self._prepare_data()

    def _prepare_data(self):
        data = []
        for url in tqdm(
            list(self.doc_meta.extraction.keys()), desc="Preparing dataset"
        ):
            metadata = self.doc_meta.extraction[url]["metadata"]

            if filter_metadata(metadata):
                continue

            metadata = task_start_token + json2token(metadata) + eos_token

            transcript = self.doc_meta.extraction[url]["transcript"]

            transcript = self.task_start_token + transcript + self.eos_token

            image_path = self.doc_meta.get_image_path(url)

            image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
            res = tv.transforms.functional.get_image_size(image_data)

            input_ids_metadata = (
                self.processor.tokenizer(
                    metadata,
                    add_special_tokens=False,
                    return_tensors="pt",
                    truncation=True,
                    padding="max_length",
                    max_length=1000,
                )["input_ids"]
                .squeeze(0)
                .long()
            )

            metadata_target = input_ids_metadata.clone()
            metadata_target[
                metadata_target == self.processor.tokenizer.pad_token_id
            ] = -100

            data.append((image_path, metadata_target, url))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, metadata, url = self.data[idx]

        image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
        image_data = image_data.permute(1, 2, 0)

        pixel_values = self.processor(image_data, return_tensors="pt").pixel_values
        pixel_values = pixel_values.squeeze()

        return {
            "pixel": pixel_values,
            "target": metadata,
            "url": url,
        }

    def get_split(self, train_ratio=0.8, val_ratio=0.1, seed=None):
        total_size = len(self.data)
        train_size = int(train_ratio * total_size)
        val_size = int(val_ratio * total_size)
        test_size = total_size - train_size - val_size  # Ensure all data is used

        indices = list(range(total_size))
        if seed is not None:
            random.Random(seed).shuffle(indices)
        else:
            random.shuffle(indices)

        train_indices = indices[:train_size]
        val_indices = indices[train_size : train_size + val_size]
        test_indices = indices[train_size + val_size :]

        train_dataset = Subset(self, train_indices)
        val_dataset = Subset(self, val_indices)
        test_dataset = Subset(self, test_indices)

        return train_dataset, val_dataset, test_dataset

In [8]:
dataset = DocMetaDataset(doc_meta, "./")

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/donut/processing_donut.py:193: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(


Preparing dataset:   0%|          | 0/4702 [00:00<?, ?it/s]

In [9]:
train, val, test = dataset.get_split(0.7, 0.28)
len(train), len(val), len(test)

(1715, 686, 49)

In [10]:
batch_size = 1
num_workers = 0

train_dataloader = DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True
)

val_dataloader = DataLoader(
    val, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

test_dataloader = DataLoader(
    test, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

In [11]:
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")

model.decoder.resize_token_embeddings(len(dataset.processor.tokenizer))
model.config.pad_token_id = dataset.processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = dataset.processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]
model.config.encoder.image_size = dataset.processor.feature_extractor.size[::-1]


# peft_config = LoraConfig(
#    lora_alpha=128, lora_dropout=0.05, r=256, bias="none", target_modules="all-linear"
# )
#
# model = get_peft_model(model, peft_config)

model = model.to("cuda")

# model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modif

In [12]:
torch.set_float32_matmul_precision("medium")

In [13]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jonasg. Use `wandb login --relogin` to force relogin


True

In [14]:
# compiled_model = torch.compile(model) # not working

In [15]:
scaler = GradScaler()  # for scaling the loss


optimizer = Prodigy(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=10 * len(train_dataloader)
)


run = wandb.init(
    # Set the project where this run will be logged
    project="donut_torch",
)


def train_one_epoch(epoch_index):
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(train_dataloader):
        batch = data

        optimizer.zero_grad()

        with autocast():

            outputs = model(
                pixel_values=batch["pixel"].to("cuda"),
                labels=batch["target"].to("cuda"),
            )

            loss = outputs.loss

        scaler.scale(loss).backward()  # why?
        scaler.step(optimizer)  # why?
        scaler.update()  # why?

        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10  # loss per batch
            print("  batch {} loss: {}".format(i + 1, last_loss))
            wandb.log({"train_loss": last_loss})
            running_loss = 0.0

        # Test loss
        if i % 200 == 190:
            running_vloss = 0
            with torch.no_grad():
                for i, vdata in enumerate(test_dataloader):
                    vbatch = vdata
                    with autocast():
                        voutputs = model(
                            pixel_values=vbatch["pixel"].to("cuda"),
                            labels=vbatch["target"].to("cuda"),
                        )
                        vloss = voutputs.loss
                    running_vloss += vloss
            avg_vloss = running_vloss / (i + 1)
            print("  vloss: {}".format(avg_vloss))
            wandb.log({"v_loss": avg_vloss})

    scheduler.step()

    return last_loss

In [16]:
# TODO: epochs too long

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.0


for epoch in range(EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    running_vloss = 0.0

    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_dataloader):
            vbatch = vdata
            with autocast():
                voutputs = model(
                    pixel_values=vbatch["pixel"].to("cuda"),
                    labels=vbatch["target"].to("cuda"),
                )
                vloss = voutputs.loss
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    wandb.log({"val_loss": avg_vloss})

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        print("Saving model ...")
        best_vloss = avg_vloss
        model_path = "model_{}_{}".format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
        wandb.save(model_path)

    epoch_number += 1

EPOCH 1:
  batch 10 loss: 15.681689357757568
  batch 20 loss: 10.561606311798096
  batch 30 loss: 8.425917053222657
  batch 40 loss: 7.112189626693725
  batch 50 loss: 5.371765279769898
  batch 60 loss: 4.589420628547669
  batch 70 loss: 4.119695711135864
  batch 80 loss: 3.9521399974822997
  batch 90 loss: 3.5171934366226196
  batch 100 loss: 3.3778762340545656
  batch 110 loss: 2.3187804222106934
  batch 120 loss: 2.315030038356781
  batch 130 loss: 2.2481177687644958
  batch 140 loss: 2.1609383702278135
  batch 150 loss: 2.2044849395751953
  batch 160 loss: 2.277420771121979
  batch 170 loss: 1.9602742552757264
  batch 180 loss: 1.9992278456687926
  batch 190 loss: 1.6106375336647034
  vloss: 2.0316522121429443
  batch 200 loss: 2.088614511489868
  batch 210 loss: 1.6877768099308015
  batch 220 loss: 1.65431307554245
  batch 230 loss: 1.5776743412017822
  batch 240 loss: 1.7079592108726502
  batch 250 loss: 1.781544601917267
  batch 260 loss: 1.5238140940666198
  batch 270 loss: 1.6

: 

# Storage and Quantization


In [15]:
from huggingface_hub import login

login()

In [16]:
# does not work when using peft LoRA
model.push_to_hub("doc_meta")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

[2024-09-25 12:10:09,338] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sodowo/doc_meta/commit/1747338c9c89360e218b1b317d12a22e270f8521', commit_message='Upload model', commit_description='', oid='1747338c9c89360e218b1b317d12a22e270f8521', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sodowo/doc_meta', endpoint='https://huggingface.co', repo_type='model', repo_id='sodowo/doc_meta'), pr_revision=None, pr_num=None)

# Storage and inference


In [21]:
task_start_token = "<s>"
eos_token = "</s>"
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
processor.tokenizer.add_special_tokens(
    {"additional_special_tokens": [task_start_token, eos_token]}
)

processor.feature_extractor.size = [2160, 3840]
model = VisionEncoderDecoderModel.from_pretrained("sodowo/doc_meta")

model.decoder.resize_token_embeddings(len(processor.tokenizer))
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(
    ["<s>"]
)[0]

preprocessor_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/71.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/809M [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/utils/import_utils.py", line 1764, in _get_module
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1140, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'transformers.models.chameleon.configuration_chameleon'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_39/2663777978.py", line 9, in <module>
    model = V

In [123]:
eval_model = model.eval()

In [131]:
pdf_path = "./attnt.pdf"
image_data = convert_from_path(pdf_path, fmt="jpeg", first_page=1, last_page=1)[0]
image_path = pdf_path.replace(".pdf", ".jpg")
image_data.save(image_path)

In [132]:
image_data = tv.io.read_image(unicodedata.normalize("NFC", image_path))
image_data = image_data.permute(1, 2, 0)

In [133]:
pixel_values = dataset.processor(image_data, return_tensors="pt").pixel_values
pixel_values = pixel_values.squeeze()

In [134]:
eval_model = eval_model.to("cuda")
with torch.no_grad():
    generated_ids = model.generate(pixel_values.unsqueeze(0).to("cuda"))

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [135]:
generated_ids

tensor([[    0,     0, 57527, 55479, 36586,  8888, 23329, 36436, 34158, 43898,
         57528, 57529, 57535, 57536, 57537, 24050, 57538, 57541, 57542,     2]],
       device='cuda:0')

In [136]:
predicted_label = dataset.processor.batch_decode(
    generated_ids, skip_special_tokens=False
)[0]

In [138]:
dataset.processor.token2json(predicted_label)

{'title': 'Attention Is All You Need',
 'website': '',
 'type': 'group',
 'phone': ''}

# Experiments
